In [5]:
%matplotlib notebook
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import CuDNNLSTM, LSTM, LeakyReLU
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
import dataloader

In [12]:
input_seq_length = 864
output_seq_length = 24
X, Y = dataloader.gen_data(input_seq_length, output_seq_length)
step_size = X.shape[1]
units = 80
second_units = 30
batch_size = 64
nb_features = X.shape[2]
epochs = 5

print(step_size, nb_features)

(131, 1728)
ci 864
B shape (131, 1728, 20)
864 20


In [11]:
model = Sequential()
model.add(CuDNNLSTM(units=units, input_shape=(step_size,nb_features),return_sequences=True))
model.add(Dropout(0.5))
model.add(CuDNNLSTM(units=second_units, return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(output_seq_length))
model.add(LeakyReLU())
model.compile(loss='mse', optimizer='adam')
model.summary()

Instructions for updating:
Use the retry module or similar alternatives.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 864, 80)           32640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 864, 80)           0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 30)                13440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                744       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 24)                0         
Total params: 46,824
Trainable params: 46,824
Non-trainable params: 0

In [10]:
model = Sequential()
model.add(LSTM(units=units, batch_input_shape=(batch_size, step_size, nb_features), stateful=True))
model.add(RepeatVector(output_seq_length))
model.add(LSTM(units=second_units, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(nb_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (64, 80)                  32320     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (64, 24, 80)              0         
_________________________________________________________________
lstm_11 (LSTM)               (64, 24, 30)              13320     
_________________________________________________________________
time_distributed_2 (TimeDist (64, 24, 20)              620       
Total params: 46,260
Trainable params: 46,260
Non-trainable params: 0
_________________________________________________________________


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_109 (InputLayer)          (16, 7, 5)           0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (16, 7, 10)          60          input_109[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o